# Importing and Displaying Data from a TSV File Using Pandas

In [15]:
import pandas as pd

# Replace 'file.tsv' with the actual path of a file
file_path = r"C:\Users\hp\Downloads\correct_twitter_201904.tsv"
# Reading the TSV file
df = pd.read_csv(file_path, sep='\t')

# Display the first few rows of the DataFrame
df.head()


,id,event,ts1,ts2,from_stream,directly_from_stream,from_search,directly_from_search,from_quote_search,directly_from_quote_search,...,retweeted,retweeted_author_id,retweeted_handle,retweeted_follower_count,mentioned_author_ids,mentioned_handles,hashtags,urls,media_keys,place_id
0,1131594960443199488,britney_201904,2022-02-28 09:34:44.627023-05:00,2022-02-28 09:34:44.627023-05:00,True,True,False,False,False,False,...,1130917791752757254,3042894016,Iesbwian,22760,None,None,None,None,None,None
1,1131594976750653440,britney_201904,2022-02-28 09:34:44.626921-05:00,2022-02-28 09:34:44.626921-05:00,True,True,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
2,1131589737955942405,britney_201904,2022-02-28 09:34:44.634058-05:00,2022-02-28 09:34:44.634058-05:00,True,True,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
3,1131594909469892610,britney_201904,2022-02-28 09:34:44.627125-05:00,2022-02-28 09:34:44.627125-05:00,True,True,False,False,False,False,...,1130917791752757254,3042894016,Iesbwian,22760,None,None,None,None,None,None
4,1131594812694511617,britney_201904,2022-02-28 09:34:44.627227-05:00,2022-02-28 09:34:44.627227-05:00,True,True,False,False,False,False,...,1130917791752757254,3042894016,Iesbwian,22760,None,None,None,None,None,None


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88037 entries, 0 to 88036
Data columns (total 46 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   id                             88037 non-null  int64 
 1   event                          88037 non-null  object
 2   ts1                            88037 non-null  object
 3    ts2                           88037 non-null  object
 4   from_stream                    88037 non-null  bool  
 5   directly_from_stream           88037 non-null  bool  
 6   from_search                    88037 non-null  bool  
 7   directly_from_search           88037 non-null  bool  
 8   from_quote_search              88037 non-null  bool  
 9   directly_from_quote_search     88037 non-null  bool  
 10  from_convo_search              88037 non-null  bool  
 11  directly_from_convo_search     88037 non-null  bool  
 12  from_timeline_search           88037 non-null  bool  
 13  d

# How many tweets were posted containing the term on each day?


In [17]:
# Convert 'created_at' to datetime, with 'utc=True' for timezone-aware data, and coerce errors to NaT
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce', utc=True)

# Check if there are any rows where 'created_at' couldn't be converted
invalid_dates = df[df['created_at'].isna()]
if not invalid_dates.empty:
    print("There are rows with invalid 'created_at' values:")
    print(invalid_dates[['created_at', 'text']])  # Display affected rows

# Function to search for a term in the 'text' column and count occurrences by date
def search_term_by_date(term, df):
    # Filter rows where the 'text' column contains the search term (case-insensitive)
    filtered_df = df[df['text'].str.contains(term, case=False, na=False)]
    
    # Remove rows with missing dates in 'created_at'
    filtered_df = filtered_df[filtered_df['created_at'].notna()]
    
    # Extract the date from 'created_at' and group by date to count occurrences
    result = filtered_df.groupby(filtered_df['created_at'].dt.date)['text'].count().reset_index()
    
    # Rename columns for clarity
    result.columns = ['date', 'tweet_count']
    
    return result

# Example usage: Search for tweets containing the term "music"
term = 'music'
result = search_term_by_date(term, df)

# Display the result
print(result)


          date  tweet_count
0   2019-03-12            3
1   2019-04-06            1
2   2019-04-14            1
3   2019-04-16            1
4   2019-04-21            1
5   2019-04-24            1
6   2019-04-26            1
7   2019-04-27            3
8   2019-04-28           22
9   2019-04-29          118
10  2019-04-30          135
11  2019-05-01           71
12  2019-05-02           72
13  2019-05-03          103
14  2019-05-04           75
15  2019-05-05           65
16  2019-05-06           71
17  2019-05-07           64
18  2019-05-08           60
19  2019-05-09           70
20  2019-05-10          307
21  2019-05-11           78
22  2019-05-12           69
23  2019-05-13           50
24  2019-05-14           67
25  2019-05-15           99
26  2019-05-16           91
27  2019-05-17          162
28  2019-05-18           61
29  2019-05-19           46
30  2019-05-20           64
31  2019-05-21          119
32  2019-05-22           80
33  2019-05-23           73
34  2019-05-24      

# How many unique users posted a tweet containing the term?


In [18]:
# Function to search for a term and count occurrences and unique users by date
def search_term_by_date(term, df):
    # Filter rows where the 'text' column contains the search term (case-insensitive)
    filtered_df = df[df['text'].str.contains(term, case=False, na=False)]
    
    # Remove rows with missing dates in 'created_at'
    filtered_df = filtered_df[filtered_df['created_at'].notna()]
    
    # Extract the date from 'created_at' and group by date
    result = filtered_df.groupby(filtered_df['created_at'].dt.date).agg(
        tweet_count=('text', 'count'),  # Count total tweets
        unique_users=('author_id', pd.Series.nunique)  # Count unique users
    ).reset_index()
    
    # Rename columns for clarity
    result.columns = ['date', 'tweet_count', 'unique_users']
    
    return result

# Example usage: Search for tweets containing the term "music"
term = 'music'
result = search_term_by_date(term, df)

# Display the result
print(result)


          date  tweet_count  unique_users
0   2019-03-12            3             3
1   2019-04-06            1             1
2   2019-04-14            1             1
3   2019-04-16            1             1
4   2019-04-21            1             1
5   2019-04-24            1             1
6   2019-04-26            1             1
7   2019-04-27            3             3
8   2019-04-28           22            21
9   2019-04-29          118            97
10  2019-04-30          135           123
11  2019-05-01           71            60
12  2019-05-02           72            63
13  2019-05-03          103            92
14  2019-05-04           75            69
15  2019-05-05           65            57
16  2019-05-06           71            57
17  2019-05-07           64            48
18  2019-05-08           60            55
19  2019-05-09           70            63
20  2019-05-10          307           299
21  2019-05-11           78            62
22  2019-05-12           69       

# How many likes did tweets containing the term get, on average?

In [19]:
# Function to search for a term and count occurrences, unique users, and average likes by date
def search_term_by_date(term, df):
    # Filter rows where the 'text' column contains the search term (case-insensitive)
    filtered_df = df[df['text'].str.contains(term, case=False, na=False)]
    
    # Remove rows with missing dates in 'created_at'
    filtered_df = filtered_df[filtered_df['created_at'].notna()]
    
    # Extract the date from 'created_at' and group by date
    result = filtered_df.groupby(filtered_df['created_at'].dt.date).agg(
        tweet_count=('text', 'count'),               # Count total tweets
        unique_users=('author_id', pd.Series.nunique),  # Count unique users
        avg_likes=('like_count', 'mean')             # Calculate average likes
    ).reset_index()
    
    # Rename columns for clarity
    result.columns = ['date', 'tweet_count', 'unique_users', 'avg_likes']
    
    return result

# Example usage: Search for tweets containing the term "music"
term = 'music'
result = search_term_by_date(term, df)

# Display the result
print(result)


          date  tweet_count  unique_users    avg_likes
0   2019-03-12            3             3  1293.666667
1   2019-04-06            1             1   210.000000
2   2019-04-14            1             1   969.000000
3   2019-04-16            1             1     6.000000
4   2019-04-21            1             1    14.000000
5   2019-04-24            1             1     8.000000
6   2019-04-26            1             1    54.000000
7   2019-04-27            3             3    65.000000
8   2019-04-28           22            21     0.954545
9   2019-04-29          118            97    20.932203
10  2019-04-30          135           123     1.074074
11  2019-05-01           71            60     3.676056
12  2019-05-02           72            63    34.708333
13  2019-05-03          103            92     2.854369
14  2019-05-04           75            69     2.840000
15  2019-05-05           65            57     3.384615
16  2019-05-06           71            57    51.070423
17  2019-0

# Where (in terms of place IDs) did the tweets come from?

In [20]:
# Function to search for a term and count occurrences, unique users, average likes, and group by place ID
def search_term_by_date_and_location(term, df):
    # Filter rows where the 'text' column contains the search term (case-insensitive)
    filtered_df = df[df['text'].str.contains(term, case=False, na=False)]
    
    # Remove rows with missing dates in 'created_at'
    filtered_df = filtered_df[filtered_df['created_at'].notna()]
    
    # Extract the date from 'created_at' and group by date and place_id
    result = filtered_df.groupby([filtered_df['created_at'].dt.date, 'place_id']).agg(
        tweet_count=('text', 'count'),              # Count total tweets
        unique_users=('author_id', pd.Series.nunique),  # Count unique users
        avg_likes=('like_count', 'mean')            # Calculate average likes
    ).reset_index()
    
    # Rename columns for clarity
    result.columns = ['date', 'place_id', 'tweet_count', 'unique_users', 'avg_likes']
    
    return result

# Example usage: Search for tweets containing the term "music"
term = 'music'
result = search_term_by_date_and_location(term, df)

# Display the result
print(result)


          date          place_id  tweet_count  unique_users    avg_likes
0   2019-03-12              None            3             3  1293.666667
1   2019-04-06              None            1             1   210.000000
2   2019-04-14              None            1             1   969.000000
3   2019-04-16              None            1             1     6.000000
4   2019-04-21              None            1             1    14.000000
..         ...               ...          ...           ...          ...
79  2019-05-29  53504716d445dcad            1             1     0.000000
80  2019-05-29              None          184           173     1.326087
81  2019-05-29  ab2f2fac83aa388d            1             1     0.000000
82  2019-05-30              None           71            65  4094.098592
83  2019-05-31              None           14            14     5.928571

[84 rows x 5 columns]


# What times of day were the tweets posted at?

In [21]:
# Function to search for a term and count occurrences, unique users, average likes, and group by time of day
def search_term_by_time_of_day(term, df):
    # Filter rows where the 'text' column contains the search term (case-insensitive)
    filtered_df = df[df['text'].str.contains(term, case=False, na=False)]
    
    # Remove rows with missing dates in 'created_at'
    filtered_df = filtered_df[filtered_df['created_at'].notna()]
    
    # Convert 'created_at' to datetime if not already done
    filtered_df['created_at'] = pd.to_datetime(filtered_df['created_at'], errors='coerce')
    
    # Extract the time from 'created_at' and group by time of day
    filtered_df['time_of_day'] = filtered_df['created_at'].dt.time
    result = filtered_df.groupby('time_of_day').agg(
        tweet_count=('text', 'count'),              # Count total tweets
        unique_users=('author_id', pd.Series.nunique),  # Count unique users
        avg_likes=('like_count', 'mean')            # Calculate average likes
    ).reset_index()
    
    # Rename columns for clarity
    result.columns = ['time_of_day', 'tweet_count', 'unique_users', 'avg_likes']
    
    return result

# Example usage: Search for tweets containing the term "music"
term = 'music'
result = search_term_by_time_of_day(term, df)

# Display the result
print(result)


     time_of_day  tweet_count  unique_users  avg_likes
0       00:00:37            1             1        0.0
1       00:00:49            1             1        1.0
2       00:00:57            1             1        0.0
3       00:01:07            1             1        0.0
4       00:01:16            1             1        0.0
...          ...          ...           ...        ...
2910    23:58:48            1             1        1.0
2911    23:58:51            1             1        1.0
2912    23:58:54            1             1        0.0
2913    23:59:24            1             1        0.0
2914    23:59:43            1             1        0.0

[2915 rows x 4 columns]


# Which user posted the most tweets containing the term?

In [22]:
# Function to search for a term and find the user who posted the most tweets
def most_active_user(term, df):
    # Filter rows where the 'text' column contains the search term (case-insensitive)
    filtered_df = df[df['text'].str.contains(term, case=False, na=False)]
    
    # Count the number of tweets for each user
    user_tweet_counts = filtered_df.groupby(['author_id', 'author_handle']).size().reset_index(name='tweet_count')
    
    # Find the user with the maximum tweet count
    most_active = user_tweet_counts.loc[user_tweet_counts['tweet_count'].idxmax()]
    
    return most_active

# Example usage: Search for tweets containing the term "music"
term = 'music'
result = most_active_user(term, df)

# Display the result
print(f"The most active user for the term '{term}' is:")
print(f"User ID: {result['author_id']}, Handle: {result['author_handle']}, Tweet Count: {result['tweet_count']}")


The most active user for the term 'music' is:
User ID: 118301422, Handle: freqnetwork, Tweet Count: 90
